In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC

# Funções auxiliares
def Adiciona_Link_em_Lista (link_atual, lista_links, categoria):
    """
    Função void que vai adicionar 'link_atual' em 'lista_links' na devida 'categoria'.
    Se o 'link_atual' já estiver dentro da 'lista_links', então verifica se a 'categoria' já está na 'lista_links'

    Param: 
        link_atual: str com o link do app
        lista_links: lista que contém os links dos apps e as categorias a que pertencem
            (ex. [[link1, [categoria1, categoria2]], [link33, [categoria3]]])
        categoria: str com a categoria que está sendo buscada

    Return:
        void
    """
    ja_registrado = False
    for registro in lista_links:
        if (link_atual in registro):
            if (categoria not in registro[1]):
                registro[1].append(categoria)
            ja_registrado = True
    
    if (not ja_registrado):
        lista_links.append([link_atual, [categoria]])

def Extrai_Apps_em_Alta (element, lista_final):
    colunas_apps = element.find_elements(By.XPATH, './/c-wiz/section/div/div[3]/div/div/div/div[1]/div')
    #print(f"Quantidade de colunas: {len(colunas_apps)}")

    for coluna in colunas_apps:
        #print(f"Coluna número: {cont}")
        apps = coluna.find_elements(By.XPATH, './div')
        #print(f"Quantidade de apps encontrados: {len(apps)}")
        for app in apps:
            try:
                classificacao = app.find_element(By.XPATH, './/div/a/div[1]/div').text
                #print(classificacao)
                link = app.find_element(By.XPATH, './/div/a').get_attribute("href")
                #print(link)
                categoria = app.find_element(By.XPATH, './/div/a/div[2]/div/div[2]/span').text
                #print(categoria)
                lista_final.append([link, categoria, classificacao])
            except:
                print("Deu ruim")

def Formata_Dimensao (leitura):
    multiplicador = 1
    numero = float(leitura[0].replace(',', '.'))
    if (leitura[1] == 'mil'):
        multiplicador *= 10**3
    elif (leitura[1] == 'mi'):
        multiplicador *= 10**6
    elif (leitura[1] == 'bi'):
        multiplicador *= 10**9
        
    return int(numero * multiplicador)

def Formata_Data(data):
    data = data.replace(".", "").split()
    dic_meses = {
        "jan":"01",
        "fev":"02",
        "mar":"03",
        "abr":"04",
        "mai":"05",
        "jun":"06",
        "jul":"07",
        "ago":"08",
        "set":"09",
        "out":"10",
        "nov":"11",
        "dez":"12"
    }
    dia = data[0]
    mes = dic_meses[data[2]]
    ano = data[-1]
    data_final = dia+'/'+mes+"/"+ano
    return data_final

In [2]:
driver = webdriver.Chrome() #Criando uma instância do Google Chrome e salvando na variável 'driver'
link_google_play = 'https://play.google.com/store/apps'
driver.get(link_google_play) #Navegando para o URL de interesse

In [3]:
#Localizando todas as divisões que contêm as categorias de Apps
xpath_element = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/c-wiz/div/c-wiz/c-wiz'
elements = driver.find_elements(By.XPATH, xpath_element)
print("Elementos localizados.")

#Procurando a divisão 'Em Alta'
lista_apps_gratuitos = []
lista_apps_rentaveis = []
lista_apps_pagos = []
for element in elements:
    try:
        titulo = element.find_element(By.XPATH, ".//c-wiz/section/header/div").text
        #print(categoria)
        if(titulo == "Em alta"):
            print("Extraindo os Apps gratuitos em alta")
            Extrai_Apps_em_Alta (element, lista_apps_gratuitos)

            botao_rentaveis = '//*[@id="ct|apps_topgrossing"]'
            driver.find_element(By.XPATH, botao_rentaveis).click()
            sleep(2)
            print("Extraindo os Apps mais rentáveis.")
            Extrai_Apps_em_Alta (element, lista_apps_rentaveis)

            botao_pagos = '//*[@id="ct|apps_topselling_paid"]'
            driver.find_element(By.XPATH, botao_pagos).click()
            sleep(2)
            print("Extraindo os Apps pagos em alta")
            Extrai_Apps_em_Alta (element, lista_apps_pagos)
    
    except:
        # print("Elemento sem categoria.")
        # print(element.text)
        pass


Elementos localizados.
Extraindo os Apps gratuitos em alta
Extraindo os Apps mais rentáveis.
Extraindo os Apps pagos em alta


In [7]:
# Após obter os links dos Apps em alta, entrar em cada link e extrair algumas informações
def Extrai_Dados (driver, lista_apps):
    lista_teste = lista_apps[:10]
    lista_geral = []
    for app in lista_teste:
        link = app[0]
        categoria = app[1]
        classificacao = app[2]
        
        driver.get(link)
        sleep(2)

        xpath_nome = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/h1/span'
        nome_app = driver.find_element(By.XPATH, xpath_nome).text

        xpath_nota = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[2]/div/div/div[1]/div[1]/div/div'
        nota_app = driver.find_element(By.XPATH, xpath_nota).text.split()[0]

        xpath_num_avaliacoes = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[2]/div/div/div[1]/div[2]'
        num_avaliacoes = driver.find_element(By.XPATH, xpath_num_avaliacoes).text.split()
        num_avaliacoes = Formata_Dimensao (num_avaliacoes)

        contem_anuncios = "Não"
        compras_no_app = "Não"
        try:
            driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]')
            content1 = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]/div/span[1]').text
            if (content1 == "Contém anúncios"):
                contem_anuncios = "Sim"
            elif (content1 == "Compras no app"):
                compras_no_app = "Sim"
            
            try:
                content2 = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]/div/span[2]').text
                if (content1 == "Contém anúncios"):
                    contem_anuncios = "Sim"
                elif (content1 == "Compras no app"):
                    compras_no_app = "Sim"
            except:
                pass
        except:
            pass

        height = 300
        driver.execute_script("window.scrollTo(0," + str(height) + ")") # Descendo um pouco a página para clicar em "Sobre este app"

        mudou_de_pagina = False
        while (not mudou_de_pagina):
            try:
                xpath_botao_infos = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[2]/div/section/header/div/div[2]/button'
                driver.find_element(By.XPATH, xpath_botao_infos).click() #Abrindo "Sobre este app"
                mudou_de_pagina = True
            except:
                height += 300
                driver.execute_script("window.scrollTo(0," + str(height) + ")") # Descendo mais um pouco a página
        sleep(1)

        informacoes = driver.find_elements(By.XPATH, '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div[3]/div')
        
        #Inicializando variáveis
        versao_app = ''
        ultima_atualizacao = ''
        android_min = ''
        num_downloads_min = ''
        classificacao_usuarios = ''
        data_lancamento = ''

        for info in informacoes:
            titulo_campo = info.find_element(By.XPATH, "./div[1]").text
            valor_campo = info.find_element(By.XPATH, "./div[2]").text
            match titulo_campo:
                case "Versão":
                    versao_app = valor_campo
                case "Atualizado em":
                    ultima_atualizacao = valor_campo
                    ultima_atualizacao = Formata_Data(ultima_atualizacao)
                case "Requer Android":
                    android_min = valor_campo.split()[0]
                case "Downloads":
                    num_downloads_min = valor_campo.split()[0].replace("+", "")
                    num_downloads_min = int(num_downloads_min.replace(".", ""))
                case "Classificação do conteúdo":
                    classificacao_usuarios = valor_campo.replace("Saiba mais", "").strip()
                case "Lançado em":
                    data_lancamento = valor_campo
                    data_lancamento = Formata_Data(data_lancamento)

        lista_geral.append([nome_app, categoria, classificacao, num_downloads_min, nota_app, num_avaliacoes, contem_anuncios, compras_no_app, versao_app, ultima_atualizacao, android_min, classificacao_usuarios, data_lancamento])
    
    colunas = ["App", "Categoria", "Ranking", "Downloads", "Nota", "Avaliacoes", "Contem_Anuncios", "Compras_no_App", "Versao", "Ultima_Atualizacao", "Android", "Classificacao_Conteudo", "Lancamento"]
    df = pd.DataFrame(lista_geral, columns=colunas)

        # print("Nome: ", nome_app)
        # print(f"Categoria: {categoria}")
        # print(f"Classificacao: {classificacao}")
        # print("Nota: ", nota_app)
        # print("Numero de avaliações: ", num_avaliacoes)
        # print("O app contém anúncios: ", contem_anuncios)
        # print("Há compras no app: ", compras_no_app)
        # print("Versao do app: ", versao_app)
        # print("Última atualização: ", ultima_atualizacao)
        # print("Android mínimo: ", android_min)
        # print("Número mínimo de Downloads: ", num_downloads_min)
        # print("Restrição de idade: ", classificacao_usuarios)
        # print("Data de lançamento: ", data_lancamento)

    return df
        

df_gratuitos = Extrai_Dados (driver, lista_apps_gratuitos)
#df_rentaveis = Extrai_Dados (driver, lista_apps_rentaveis)
#df_pagos = Extrai_Dados (driver, lista_apps_pagos)

In [8]:
df_gratuitos
#Adicionar duas colunas "Pago", "Preco"

,App,Categoria,Ranking,Downloads,Nota,Avaliacoes,Contem_Anuncios,Compras_no_App,Versao,Ultima_Atualizacao,Android,Classificacao_Conteudo,Lancamento
0,Shopee: Compre de Tudo Online,Compras,1,100000000,"4,5",3360000,Não,Não,2.91.31,11/08/2022,4.4,Classificação Livre,20/10/2019
1,SHEIN-Compras de Moda Online,Compras,2,100000000,"4,8",4099999,Não,Não,8.3.8,4/08/2022,5.0,"Classificação 14 anos • Conteúdo Sexual, Lingu...",18/03/2014
2,Scret: perguntas anônimas,Social,3,1000000,"3,6",5870,Sim,Não,2.4,9/08/2022,5.0,Classificação Livre,15/07/2022
3,"Nubank: conta, cartão e mais",Finanças,4,50000000,"4,7",1920000,Não,Não,Varia de acordo com o dispositivo,5/08/2022,Varia,Classificação Livre,25/04/2014
4,MegaFlix - Filmes e Séries,Entretenimento,5,1000000,"4,6",48300,Sim,Não,2,21/07/2022,4.4,"Classificação 12 anos • Violência, Linguagem I...",21/07/2022
5,Auxílio Brasil GovBR,Social,6,10000000,"3,5",146000,Não,Não,3.23.0,2/08/2022,5.0,Classificação Livre,23/09/2015
6,CAIXA Tem,Finanças,7,100000000,"4,1",4710000,Não,Não,Varia de acordo com o dispositivo,1/08/2022,Varia,Classificação Livre,6/04/2020
7,phone cleaner,Ferramentas,8,10000000,"3,7",7340,Sim,Não,1.5.1,9/07/2022,7.0,Classificação Livre,29/04/2022
8,Bravo Cleaner: Speed Booster,Ferramentas,9,10000000,"4,4",40000,Sim,Não,1.3.4.1001,4/08/2022,5.0,Classificação Livre,21/09/2021
9,Kwai - Ver Vídeos Bacanas,Reproduzir e editar vídeos,10,100000000,"4,8",9410000,Sim,Não,6.7.10.526806,12/08/2022,5.0,Classificação 12 anos • Linguagem Imprópria,2/03/2018


In [5]:
#Tudo isso vai dentro de Extrai_Dados
lista_geral = []
app1 = lista_apps_gratuitos[11]
link = app1[0]
categoria = app1[1]
classificacao = app1[2]

driver.get(link)
sleep(2)

xpath_nome = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/h1/span'
nome_app = driver.find_element(By.XPATH, xpath_nome).text

xpath_nota = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[2]/div/div/div[1]/div[1]/div/div'
nota_app = driver.find_element(By.XPATH, xpath_nota).text.split()[0]

xpath_num_avaliacoes = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[2]/div/div/div[1]/div[2]'
num_avaliacoes = driver.find_element(By.XPATH, xpath_num_avaliacoes).text.split()
num_avaliacoes = Formata_Dimensao (num_avaliacoes)

contem_anuncios = "Não"
compras_no_app = "Não"

try:
    driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]')
    content1 = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]/div/span[1]').text
    if (content1 == "Contém anúncios"):
        contem_anuncios = "Sim"
    elif (content1 == "Compras no app"):
        compras_no_app = "Sim"
    
    try:
        content2 = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]/div/span[2]').text
        if (content1 == "Contém anúncios"):
            contem_anuncios = "Sim"
        elif (content1 == "Compras no app"):
            compras_no_app = "Sim"
    except:
        pass
except:
    pass

height = 300
driver.execute_script("window.scrollTo(0," + str(height) + ")") # Descendo um pouco a página

mudou_de_pagina = False
while (not mudou_de_pagina):
    try:
        xpath_botao_infos = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[2]/div/section/header/div/div[2]/button'
        driver.find_element(By.XPATH, xpath_botao_infos).click() #Abrindo "Sobre este app"
        mudou_de_pagina = True
    except:
        height += 300
        driver.execute_script("window.scrollTo(0," + str(height) + ")") # Descendo mais um pouco a página
sleep(1)

informacoes = driver.find_elements(By.XPATH, '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div[3]/div')
#Inicializando variáveis
versao_app = ''
ultima_atualizacao = ''
android_min = ''
num_downloads_min = ''
classificacao_usuarios = ''
data_lancamento = ''

for info in informacoes:
    titulo_campo = info.find_element(By.XPATH, "./div[1]").text
    valor_campo = info.find_element(By.XPATH, "./div[2]").text
    match titulo_campo:
        case "Versão":
            versao_app = valor_campo
        case "Atualizado em":
            ultima_atualizacao = valor_campo
            ultima_atualizacao = Formata_Data(ultima_atualizacao)
        case "Requer Android":
            android_min = valor_campo.split()[0]
        case "Downloads":
            num_downloads_min = valor_campo.split()[0].replace("+", "")
            num_downloads_min = int(num_downloads_min.replace(".", ""))
        case "Classificação do conteúdo":
            classificacao_usuarios = valor_campo.replace("Saiba mais", "").strip()
        case "Lançado em":
            data_lancamento = valor_campo
            data_lancamento = Formata_Data(data_lancamento)

lista_geral.append([nome_app, categoria, classificacao, nota_app, num_avaliacoes, contem_anuncios, compras_no_app, versao_app, ultima_atualizacao, android_min, num_downloads_min, classificacao_usuarios, data_lancamento])
print("Nome: ", nome_app)
print(f"Categoria: {categoria}")
print(f"Classificacao: {classificacao}")
print("Nota: ", nota_app)
print("Numero de avaliações: ", num_avaliacoes)
print("O app contém anúncios: ", contem_anuncios)
print("Há compras no app: ", compras_no_app)
print("Versao do app: ", versao_app)
print("Última atualização: ", ultima_atualizacao)
print("Android mínimo: ", android_min)
print("Número mínimo de Downloads: ", num_downloads_min)
print("Restrição de idade: ", classificacao_usuarios)
print("Data de lançamento: ", data_lancamento)

Nome:  gov.br
Categoria: Ferramentas
Classificacao: 12
Nota:  4,7
Numero de avaliações:  2310000
O app contém anúncios:  Não
Há compras no app:  Não
Versao do app:  3.2.35
Última atualização:  24/06/2022
Android mínimo:  6.0
Número mínimo de Downloads:  10000000
Restrição de idade:  Classificação Livre
Data de lançamento:  27/04/2020
